In [ ]:
import tensorflow as tf

import os, pathlib, warnings, urllib, shutil

warnings.filterwarnings("ignore")

In [ ]:
giturl = "https://github.com/dicodingacademy/assets/releases/download/release/"

dataset = "rockpaperscissors.zip"

urllib.request.urlretrieve(os.path.join(giturl, dataset), dataset)

shutil.unpack_archive('rockpaperscissors.zip', '/content', 'zip')

In [ ]:
root = "/content/rockpaperscissors/rps-cv-images"

os.listdir(root)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

rescale = 1.0 / 255.0

trainset = ImageDataGenerator(
  rescale = rescale,
  fill_mode = 'nearest'
)

validset = ImageDataGenerator(rescale=rescale)

In [ ]:
total_batch = 128

train = trainset.flow_from_directory(
  root,
  target_size = (150, 150),
  class_mode = "categorical",
  batch_size = total_batch,
  subset = "training"
)

In [ ]:
labeler = list(train.class_indices.keys())

labeler

In [ ]:
valid = validset.flow_from_directory(
  root,
  target_size = (150, 150),
  class_mode = "categorical",
  batch_size = total_batch,
  subset = "validation"
)

In [ ]:
from tensorflow.keras.models import Sequential

from tensorflow.keras import layers

model = Sequential([
  layers.Conv2D(16, 3, activation="relu", input_shape=(150, 150, 3)),
  layers.MaxPool2D((2, 2)),
  layers.Conv2D(32, 3, activation="relu"),
  layers.MaxPool2D((2, 2)),
  layers.Conv2D(64, 3, activation="relu"),
  layers.MaxPool2D((2, 2)),
  layers.Flatten(),
  layers.Dropout(0.3),
  layers.Dense(128, activation="relu"),
  layers.Dense(3, activation="softmax")
])

loss = "categorical_crossentropy"

model.compile(optimizer="adam", loss=loss, metrics=["accuracy"])

In [ ]:
hist = model.fit(train, epochs=10, validation_data=valid)